## Exercise 2: Merging and Deriving New Columns

Skills: 
* Merge 2 dataframes
* F-strings!
* Markdown cells
* Build on groupby/agg knowledge, derive new columns, exporting
* Practice committing on GitHub

References: 
* https://docs.calitp.org/data-infra/analytics_new_analysts/01-data-analysis-intro.html
* https://docs.calitp.org/data-infra/analytics_tools/saving_code.html

In [ ]:
import pandas as pd

Use of f-strings. [Read more on this](https://realpython.com/python-f-strings/#f-strings-a-new-and-improved-way-to-format-strings-in-python).

Also, click on this Markdown cell and see how to do different formatting syntax within Markdown. [Reference this](https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook).

If you don't have access to Google Cloud Storage, change the path to pull from our truncated sample parquets stored in the repo.

We use [relative paths](https://towardsthecloud.com/get-relative-path-python) rather than absolute paths. Since we are in the `starter_kit` directory, we just need to go one more level in to the `data` subfolder. To get one level outside of `starter_kit`, use `../` and you'll end up in `data-analyses`. 

```
FOLDER = "./data/"
FILE_NAME = "exercise_2_3_ntd_metrics_2019.parquet"
df = pd.read_parquet(f"{FOLDER}{FILE_NAME}")

```

In [ ]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/bus_service_increase/"
FILE_NAME = "ntd_metrics_2019.csv"

df = pd.read_csv(f"{GCS_FILE_PATH}{FILE_NAME}")
df.head(2)

In [ ]:
FILE_NAME = "ntd_vehicles_2019.csv"
df2 = pd.read_csv(f"{GCS_FILE_PATH}{FILE_NAME}")

df2.head(2)

In [ ]:
len(df)

In [ ]:
df["NTD ID"].nunique()

In [ ]:
len(df2)
df2["NTD ID"].nunique()

### To do:

* Start with the `ntd_metrics_2019.csv` dataset.
* Merge in the `ntd_vehicles_2019.csv` dataset from the same location within the GCS bucket, but only keep a couple of columns.
* Print out what states there are using `value_counts`
* Subset and only keep the following states: NY, CA, TX, ID, MS
* Calculate some aggregate statistics grouping by states (the point of the exercise is to aggregate, less so on whether the stats make sense):
    * Include: sum, mean, count (of operators), nunique (of city)
    * Challenge: give a per capita measure, such as total service vehicles per 100,000 residents
* Plot the per capita measure across the 5 states (some states are very populous and some are not...per capita hopefully normalizes pop differences)

In [ ]:
list(df.columns)

In [ ]:
df.Agency.value_counts()

### Helpful Hints and Best Practices

* Start with comprehensive approach in writing down all the lines of code to clean data. 
* Once the data cleaning process is done, work on refining the code and tidying it to see what steps can be chained together, what steps are done repeatedly (use a function!), etc.

#### Chaining
Similar to **piping** in R, where you can pipe multiple operations in 1 line of code with `>>`, you can do a similar method of chaining in Python. There is also a `df.pipe` function, but that's slightly different.

Make use of parentheses to do this. Also, use `df.assign` (see below) so you don't run into the `SettingWithCopyWarning`, which may pop up if you decide to subset your data. 

#### Assign 
You can create new columns in place, and the warning that comes up is mostly harmless. But, `assign` also lets you chain more operations after. [More clarification.](https://pythonguides.com/add-a-column-to-a-dataframe-in-python-pandas/)
```
states_clean = (states_clean
    # Assign is similar to R dplyr's mutate
    .assign(
        # Strip leading or trailing blanks (slightly different than replace)
        # Decide if you want to replace all blanks or just leading/trailing
        Agency = (states_clean.Agency.str.strip()
                .str.replace('(', '').str.replace(')', '')
        ),
        # Do something similar for City as above
        City = states_clean.City.str.strip(),
        # Replace blanks with nothing
        State = states_clean.State.str.replace(' ', '')
    ).astype({
        "Population": int, 
        "Fare_Revenues": int,
    })
)
```

Alternatively, try it with a loop:

```
for c in ["Agency", "City"]:
    df[c] = (df[c].str.strip()
            .str.replace('(', '')
            .str.replace(')', '')
            .astype(int)
            )
```

#### Using `str.contains` with some special characters
Use backslash `\` to "escape". [StackOverflow explanation](https://stackoverflow.com/questions/48699907/error-unbalanced-parenthesis-while-checking-if-an-item-presents-in-a-pandas-d)
`states_clean[states_clean.Fare_Revenues.str.contains("\(")]`


#### Merging
If your merge results produces a `col_x` and `col_y`, add more columns to your list of merge columns, with `on = ["col1", "col2"]`.

#### Use `isin` to filter by multiple conditions

```
keep_me = ["CA", "NY", "TX"]
df2 = df[df.State.isin(keep_me)]
```